Find the residuals between ICESat-2 elevation measurements and a reference elevation (taken from IceBridge ATM flyover of Zachariae Isstrom)

Taryn Black, ICESat-2 Hackweek, June 17-21 2019

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# Set data location info

In [2]:
home_dir = "/home/jovyan/xtrak/"
cross_file = "data_prod/Intersections_ATM20180418.csv"

# Load ICESat-2 crossover points

ICESat-2 track points that were identified as crossing our OIB ATM reference track in Intersections.ipynb. This file contains both the ICESat-2 elevation data and the OIB ATM data at the crossover points.

In [3]:
cross_df = pd.read_csv(home_dir + cross_file, parse_dates=[4])

cross_df.head()
#cross_df.info()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r


# Visual check of data

Plot reference elevation profile and crossover data points.

In [4]:
%matplotlib widget

plt.figure(figsize=(12,8))
ax1 = plt.scatter(cross_df['dist_along']/1000, cross_df['ATM_elev'], c='black')
ax2 = plt.scatter(cross_df['dist_along']/1000, cross_df['z_ATL06'], c=cross_df['t_ATL06'], s=12)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')
plt.title('Elevation profiles from ATM and ICESat-2')
plt.colorbar(label='Time')
plt.legend(['Reference track']);

FigureCanvasNbAgg()

# Calculate residual between ICESat-2 track crossovers and reference elevation profile

For each point in the ICESat-2 crossover dataframe, subtract the reference elevation at the same distance along-track.

$residual = z_{ICESat2} - z_{reference}$

Thus, positive values indicate an increase in elevation compared to the reference track, and negative values indicate a decrease in elevation.

In [5]:
cross_df['residuals'] = cross_df['z_ATL06'] - cross_df['ATM_elev']

cross_df.head()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,residuals
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l,-0.904243
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r,-0.962390
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r,-21.519910
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l,-1.152050
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r,5.487244


Let's plot the residuals!

In [12]:
plt.figure(figsize=(12,4))
plt.scatter(cross_df['dist_along']/1000, cross_df['residuals'], c=cross_df['t_ATL06'])
plt.axhline(0, color='k', lw=0.5)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')
plt.title('Elevation residuals \n (+)=raised, (-)=lowered')
plt.colorbar(label='Time');

FigureCanvasNbAgg()

# Export dataframe to CSV

Exported CSV file includes distance along track, ATM data (elevation, index), ATL06 data (elevation, index, time, groundtrack), and the residual (difference between ATL06 and ATM).

In [14]:
cross_df.to_csv(home_dir + "data_prod/residuals.csv")

# Residual statistics

Let's further explore what's going on in our elevation data.

## Variation in elevation changes, along-track

Bin the residuals by distance along track, and look at the standard deviation of residual values in each bin, using a box plot. Why do this? We hypothesize that we will see more elevation variation near the terminus than farther up-flow.

In [34]:
bins = np.arange(50,140,10)
print(bins)
cross_df['dist_binned'] = pd.cut(cross_df['dist_along']/1000, bins)
cross_df.head()

[ 50  60  70  80  90 100 110 120 130]


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,residuals,dist_binned
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l,-0.904243,"(120, 130]"
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r,-0.962390,"(120, 130]"
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r,-21.519910,"(90, 100]"
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l,-1.152050,"(100, 110]"
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r,5.487244,"(100, 110]"


In [35]:
cross_df.describe()

,dist_along,ATM_elev,idx_ATM,z_ATL06,idx_ATL06,residuals
count,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000
mean,95537.751622,142.674674,2557.500000,149.744321,365132.760638,7.069647
std,19736.138045,143.488752,552.391258,203.396865,205362.454985,128.404198
min,56862.936846,27.339600,1513.000000,26.104408,1578.000000,-60.585456
25%,80521.837482,37.727925,2133.000000,35.312495,193376.250000,-4.835666
50%,95369.899221,86.165900,2554.000000,76.361470,374275.500000,-1.262035
75%,112350.591643,251.812775,3040.000000,246.346338,541517.000000,0.203516
max,127480.010096,570.238000,3468.000000,2154.631000,716129.000000,1746.045800


In [36]:
cross_df.boxplot(by='dist_binned', column='residuals')

FigureCanvasNbAgg()